In [1]:
import pickle
from pathlib import Path
import numpy as np
import xarray as xr

In [2]:
output_dir = Path(r"C:\Users\SPEEM\Documents\Python Packages\speem\out") / "fixed"


def fix_data(filepath: Path, new_filepath: Path):
    with open(filepath, "rb") as f:
        raw_data: xr.DataArray = pickle.load(f)
    raw_counts: np.ndarray = raw_data["detector-frame-data"].data
    if raw_counts.ndim == 1:
        raw_counts = np.concatenate(raw_counts)
    else:
        raw_counts = np.concatenate(raw_counts, -1).transpose()
    terminal_voltages = raw_data["power_supply-terminal_voltages-data"].data[0]
    n_counts = raw_counts.shape[0]
    da = xr.DataArray(
        data=raw_counts,
        coords={
            "counts": np.linspace(0, n_counts, n_counts),
            "dim": ["x_px", "y_px", "t_px"],
        },
        dims=("counts", "dim"),
        attrs=terminal_voltages,
    )

    if not new_filepath.parent.exists():
        new_filepath.parent.mkdir()
    da.to_netcdf(new_filepath.with_suffix(".nc"))
    return da, raw_data

In [4]:
filepath = r"C:\Users\SPEEM\Documents\Python Packages\speem\out\data\jgobbo\2025-09-25-BiSe bands pls-0\pickle\raw_daq.pickle"
da, raw_data = fix_data(filepath, output_dir / "2025-09-25" / "01.nc")

In [3]:
filepath = r"C:\Users\SPEEM\Documents\Python Packages\speem\out\data\jgobbo\2025-09-25-BiSe bands pls-0_09-25-31-323289\pickle\raw_daq.pickle"
with open(filepath, "rb") as f:
    raw = pickle.load(f)

In [ ]:
import numpy as np
import xarray as xr

raw: xr.Dataset
trimmed = raw.copy(deep=True)

for data_var in raw.data_vars:
    dtype = trimmed[data_var].dtype
    if dtype == "object":
        bad_type = not isinstance(trimmed[data_var].data[0], (float, int, np.ndarray))
        if bad_type:
            print(f"dropping {data_var}")
            trimmed = trimmed.drop_vars(data_var)
        else:
            actual_type = type(trimmed[data_var].data[0])
            print(f"changing {data_var} to {actual_type}")
            trimmed[data_var] = trimmed[data_var].astype(actual_type)

for data_var in trimmed.data_vars:
    var = trimmed[data_var]
    if var.dtype == "object":
        data = np.concatenate(var.data)
        trimmed[data_var] = xr.DataArray(data)

trimmed.to_netcdf(r"C:\Users\SPEEM\Desktop\test.nc")

dropping power_supply-terminal_voltages-data
changing detector-timing_delay-data to <class 'float'>
changing detector-frame_time-data to <class 'float'>
changing phony-stages-0-data to <class 'int'>
changing detector-frame-data to <class 'numpy.ndarray'>


In [24]:
data = trimmed["detector-frame-data"].data  # = np.array([[0, 0, 0]], dtype=np.float64)
data[0] = np.array([[0, 0, 0]], dtype=np.float64)
trimmed["detector-frame-data"].data = data

In [25]:
trimmed.to_netcdf(r"C:\Users\SPEEM\Desktop\test.nc")

ValueError: setting an array element with a sequence.

In [2]:
import xarray as xr

test_ds = xr.load_dataset(
    r"C:\Users\SPEEM\Documents\Python Packages\speem\out\data\jgobbo\2025-09-25-BiSe bands pls-0_15-07-21-636827\nc\raw_daq.nc"
)

In [4]:
test_ds["detector-frame-data"]

<xarray.DataArray 'detector-frame-data' (dim_0: 15289, dim_1: 3)>
array([[-12.07620471,  -8.1045947 , 299.193815  ],
       [  6.49084796,   7.01083302, 296.276624  ],
       [  5.56742622,  -5.71087038, 297.296048  ],
       ...,
       [-10.04109077,  -9.05491222, 296.326685  ],
       [ -6.93911095,  10.7224505 , 302.998451  ],
       [-10.64176317,  11.90586477, 299.116448  ]])
Dimensions without coordinates: dim_0, dim_1